# palaestrAI in a Jupyter Notebook

This is just the equivalent of a system test for the random action experiment.

## Imports

In [1]:
import palaestrai

If the above line works without error, at least the import is done. Thats a first good starter. Now let's import the rest…

In [2]:
import os
import pprint
import tempfile
from pathlib import Path

## Setting the Runtime Configuration

Usually, this is not necessary like this. But because we're running a self-contained test, we want to have a new, fresh database where we want to put it. Which is, in a temporary directory.

In [3]:
store_dir = tempfile.TemporaryDirectory()
store_dir

<TemporaryDirectory '/tmp/tmppc_kbk1y'>

We're now going to change palaestrAI's runtime configuration to point to the new directory.

In [4]:
from palaestrai.core import RuntimeConfig

runtime_config = RuntimeConfig()
runtime_config.reset()
runtime_config.load({"store_uri": "sqlite:///%s/palaestrai.db" % store_dir.name})
pprint.pprint(runtime_config.to_dict())

{'executor_bus_port': 4242,
 'logger_port': 4243,
 'logging': {'filters': {'debug_filter': {'()': 'palaestrai.core.runtime_config.DebugLogFilter'}},
             'formatters': {'debug': {'format': '%(asctime)s '
                                                '%(name)s[%(process)d]: '
                                                '%(levelname)s - %(message)s '
                                                '(%(module)s.%(funcName)s in '
                                                '%(filename)s:%(lineno)d)'},
                            'simple': {'format': '%(asctime)s '
                                                 '%(name)s[%(process)d]: '
                                                 '%(levelname)s - '
                                                 '%(message)s'}},
             'handlers': {'console': {'class': 'logging.StreamHandler',
                                      'formatter': 'simple',
                                      'level': 'INFO',
                  

## Create the Database

Next, we create the database at the given URI. It will complain that we're not using TimescaleDB, which is okay.

In [5]:
from palaestrai.store.database_util import setup_database
setup_database(runtime_config.store_uri)

Could not create extension timescaledb and create hypertables: (sqlite3.OperationalError) near "EXTENSION": syntax error
[SQL: CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;]
(Background on this error at: https://sqlalche.me/e/14/e3q8). Your database setup might lead to noticeable slowdowns with larger experiment runs. Please upgrade to PostgreSQL with TimescaleDB for the best performance.


In [6]:
assert Path("%s/palaestrai.db" % store_dir.name).is_file()

## Run Experiment

In this part, we load our dummy experiment and run it.

In [7]:
experiment_file_path = Path().absolute() / '..' / 'fixtures' / 'dummy_run.yml'
assert experiment_file_path.is_file()

In [8]:
rc = palaestrai.execute(str(experiment_file_path))
assert rc[1].value == 4
rc

(['Yo-ho, a dummy experiment run for me!'], <ExecutorState.EXITED: 4>)

If you see something like `('Yo-ho, a dummy experiment run for me!', <ExecutorState.EXITED: 4>)` as output of the previous line, then congratulations, everything went well! Now onwards to the final step…

## Verify Data from the Store

Now that an experiment has been run, there should be something in the store. Note that we don't check whether something *meaningful* is in the store, only that there is *something* in the store.

In [9]:
from sqlalchemy import select

import palaestrai.store
import palaestrai.store.database_model as dbm

In [10]:
dbh = palaestrai.store.Session()

In [11]:
q = select(dbm.Experiment)
str(q)

'SELECT experiments.id, experiments.name, experiments.document, experiments.document_json \nFROM experiments'

In [12]:
experiment = dbh.execute(q).first()[dbm.Experiment]
experiment.name

'Dummy Experiment record for ExperimentRun 818bb3e1-93b4-41db-bc03-41428f43e787'

In [13]:
our_experiment_run = experiment.experiment_runs[0]
str(our_experiment_run)

'<ExperimentRun(id=1, uid="Yo-ho, a dummy experiment run for me!", experiment_id=1, document=<palaestrai.experiment.experiment_run.ExperimentRun object at 0x7f1717bdb880>>'

The `document` property of the experiment run object should contain the YAML file, and we should be able to de-searialize it. Let's check that.

In [14]:
assert our_experiment_run.document

In [15]:
our_experiment_run._document_json

{'py/object': 'palaestrai.experiment.experiment_run.ExperimentRun',
 'seed': 42,
 'uid': 'Yo-ho, a dummy experiment run for me!',
 'version': '3.4',
 'schedule_config': [{'phase_0': {'environments': [{'environment': {'name': 'palaestrai.environment.dummy_environment:DummyEnvironment',
       'uid': 'myenv',
       'params': {'discrete': True}}}],
    'agents': [{'name': 'mighty_defender',
      'brain': {'name': 'palaestrai.agent.dummy_brain:DummyBrain',
       'params': {'store_path': './custom'}},
      'muscle': {'name': 'palaestrai.agent.dummy_muscle:DummyMuscle',
       'params': {}},
      'objective': {'name': 'palaestrai.agent.dummy_objective:DummyObjective',
       'params': {'params': 1}},
      'sensors': ['myenv.0', 'myenv.1', 'myenv.2', 'myenv.3', 'myenv.4'],
      'actuators': ['myenv.0', 'myenv.1', 'myenv.2', 'myenv.3', 'myenv.4']},
     {'name': 'evil_attacker',
      'brain': {'name': 'palaestrai.agent.dummy_brain:DummyBrain',
       'params': {}},
      'muscle': {'na

In [16]:
our_experiment_run.experiment_run_instances[0].uid

'007198d8-4e08-40a0-83c5-1bd2b12d1201'

… and so on. For a system test, this is enough.